In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import importlib
import sys
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
if "../" not in sys.path:
    sys.path.append("../")
import appUtilities as au

In [3]:
df = pd.read_pickle('../data/sample_data.pkl')

In [4]:
df.head()

,Date,Item,Type,Subtype,Location,Payment Method,Amount,ShopBack?,Notes,month,week_start
0,2023-03-17,Prawn Noodles,Dining,Lunch,Work,PayLah,7.5,No,NaN,2023-03-01,2023-03-13
1,2023-03-17,Poke Bowl,Dining,Dinner,Novena,GrabPay,14.5,Yes,NaN,2023-03-01,2023-03-13
2,2023-03-17,Bread,Dining,Breakfast,Barcook,PayLah,3.0,No,NaN,2023-03-01,2023-03-13
3,2023-03-18,Car Wash,Transport,Private Transport,SPC Novena,Cash,12.0,No,NaN,2023-03-01,2023-03-13
4,2023-03-18,Thunder Tea Rice,Dining,Lunch,Vivo,GrabPay,16.0,Yes,YZ,2023-03-01,2023-03-13


In [5]:
tdf = au.get_dates(df, start_date=au.get_today(), end_date=au.get_today())

In [6]:
# Get data for the month of october
month_df = au.get_month(df, month='2023-10-01')
# Get data for onw week in october
week_df = au.get_week(df, week='2023-10-23')

# Pie Chart

Generate a pie chart at the `Type` level for the given period


In [7]:
sum_df = month_df.groupby('Type').agg(
    total=pd.NamedAgg(column='Amount', aggfunc='sum')
).reset_index().sort_values(by='Type')
sum_df

,Type,total
0,Dining,835.62
1,Healthcare,14.00
2,Leisure,115.00
3,Shopping,663.81
4,Transport,29.22
5,Travel,476.70


In [15]:
# Create a donut chart using Plotly Graph Objects
fig = go.Figure()
values = sum_df.total.values
labels = sum_df.Type.values
fig.add_trace(go.Pie(
    labels=labels,
    values=values,
    hole=0.5,  # Set the hole parameter to create a donut chart
    marker=dict(colors=['red', 'green', 'blue', 'orange'])  # You can customize colors if needed
))
total_spending = sum(values)
fig.update_layout(
    # title='Donut Chart Example',
    annotations=[dict(text=f'${total_spending:,.0f}', x=0.5, y=0.5, font_size=20, showarrow=False)]
    # , legend= dict(x = 0.5, y=-0.1, orientation='h')
)

fig.show()

# Monthly / weekly Comparison

Comparing spending of main types over the last `n` months / weeks

In [47]:
# How many months to take
n_months = 4
# Get the first day of the current month
current_month = datetime.date.today().replace(day=1)
# Get the first day of the month for the last n months (this month included)
last_n_months = list(map(lambda x: (current_month-relativedelta(months=x)), range(0, n_months)))
# Get the relevant entries (entries in the last n_months)
n_months_df=df.loc[df.month.isin(last_n_months)]

In [55]:
# Aggregate the spending at the type-month level
agg_df=n_months_df.groupby(['Type', 'month']).agg(
    total = pd.NamedAgg(column='Amount', aggfunc='sum')
).reset_index()

agg_df.dtypes

Type             object
month    datetime64[ns]
total           float64
dtype: object

In [70]:
# Cycle colors
color_mapper = ['red', 'green', 'blue', 'orange']
# Get the types
types = sorted(agg_df.Type.unique())
# Plotting the figure
fig = go.Figure()
# Track color cycle
color_id = 0
for group, tdf in agg_df.groupby("Type"):
    color = color_mapper[color_id%len(color_mapper)]
    fig.add_trace(
        go.Scatter(
            x = tdf.month.values
            , y= tdf.total.values
            , mode='lines+markers'
            , name=group
            , line=dict(color=color, width=2)
            , marker=dict(symbol='diamond', size=8)

        )
    )
    color_id+=1
    # Customize layout
    fig.update_layout(
        title='Prices of Items Over the Past Few Months',
        xaxis=dict(title='Month'),
        yaxis=dict(title='Price'),
        # legend=dict(x=0.02, y=0.98),
        margin=dict(l=50, r=50, t=50, b=50)
    )    

In [71]:
fig.show()

In [60]:
group

'Travel'

# Calendar Plot

[Plotly API reference](https://plotly.com/python/heatmaps/#:~:text=12-,Heatmap%20with%20Datetime%20Axis,-import%20plotly.graph_objects)

[Existing package](https://github.com/brunorosilva/plotly-calplot)